# Tutorial on CMU-Multimodal SDK

This is a tutorial on using ***CMU-Multimodal SDK*** to load and process multimodal time-series datasets and training a simple late-fusion LSTM model on the processed data. 

For this tutorial, we specify some constants in `./constans/paths.py`. Please first take a look and modify the paths to point to the correct folders.

## Downloading the data

We start off by (down)loading the datasets. In the SDK each dataset has three sets of content: `highlevel`, `raw` and `labels`. `highlevel` contains the extracted features for each modality (e.g OpenFace facial landmarks, openSMILE acoustic features) while `raw` contains the raw transctripts, phonemes. `labels` are self-explanatory. Note that some datasets have more than just one set of annotations so `labels` could also give you multiple files.

Currently there's a caveat that the SDK will not automatically detect if you have downloaded the data already. In event of that it will throw a `RuntimeError`. We work around that by `try/except`. This is not ideal but it will work for now.

In [17]:
from constants import SDK_PATH, DATA_PATH, WORD_EMB_PATH, CACHE_PATH
import sys

if SDK_PATH is None:
    print("SDK path is not specified! Please specify first in constants/paths.py")
    exit(0)
else:
    sys.path.append(SDK_PATH)

import mmsdk
import os
import re
import numpy as np
from mmsdk import mmdatasdk as md
from subprocess import check_call, CalledProcessError

# create folders for storing the data
if not os.path.exists(DATA_PATH):
    check_call(' '.join(['mkdir', '-p', DATA_PATH]), shell=True)

# download highlevel features, low-level (raw) data and labels for the dataset MOSI
# if the files are already present, instead of downloading it you just load it yourself.
# here we use CMU_MOSI dataset as example.

DATASET = md.cmu_mosi

try:
    md.mmdataset(DATASET.highlevel, DATA_PATH)
except RuntimeError:
    print("High-level features have been downloaded previously.")

try:
    md.mmdataset(DATASET.raw, DATA_PATH)
except RuntimeError:
    print("Raw data have been downloaded previously.")
    
try:
    md.mmdataset(DATASET.labels, DATA_PATH)
except RuntimeError:
    print("Labels have been downloaded previously.")

High-level features have been downloaded previously.
Raw data have been downloaded previously.
Labels have been downloaded previously.


## Inspecting the downloaded files

We can print the files in the target data folder to see what files are there.

We can observe a bunch of files ending with `.csd` extension. This stands for ***computational sequences***, which is the underlying data structure for all features in the SDK. We will come back to that later when we load the data. For now we just print out what computational sequences we have downloaded.

In [18]:
# list the directory contents... let's see what features there are
data_files = os.listdir(DATA_PATH)
print('\n'.join(data_files))

CMU_MOSI_ModifiedTimestampedWords.csd
CMU_MOSI_OpenSmile_EB10.csd
CMU_MOSI_openSMILE_IS09.csd
CMU_MOSI_Opinion_Labels.csd
CMU_MOSI_TimestampedWords.csd
CMU_MOSI_VisualFacet_4.1.csd
CMU_MOSI_OpenFace2.csd
CMU_MOSI_TimestampedPhones.csd
CMU_MOSI_COVAREP.csd


## Loading a multimodal dataset

Loading the dataset is as simple as telling the SDK what are the features you need and where are their computational sequences. You can construct a dictionary with format `{feature_name: csd_path}` and feed it to `mmdataset` object in the SDK.

In [19]:
# define your different modalities - refer to the filenames of the CSD files
visual_field = 'CMU_MOSI_VisualFacet_4.1'
acoustic_field = 'CMU_MOSI_COVAREP'
text_field = 'CMU_MOSI_ModifiedTimestampedWords'


features = [
    text_field, 
    visual_field, 
    acoustic_field
]

recipe = {feat: os.path.join(DATA_PATH, feat) + '.csd' for feat in features}
dataset = md.mmdataset(recipe)

  0%|          | 0/93 [00:00<?, ? Computational Sequence Entries/s]

<Success>: Computational sequence read from file ./data/CMU_MOSI_ModifiedTimestampedWords.csd ...
<Status>: Checking the integrity of the data in <CMU_MOSI_ModifiedTimestampedWords> computational sequence ...
<Success>: <CMU_MOSI_ModifiedTimestampedWords> computational sequence data in correct format.
<Status>: Checking the integrity of the metadata in <CMU_MOSI_ModifiedTimestampedWords> computational sequence ...
<Success>: <CMU_MOSI_ModifiedTimestampedWords> computational sequence metadata in correct format
<Success>: Computational sequence read from file ./data/CMU_MOSI_VisualFacet_4.1.csd ...
<Status>: Checking the integrity of the data in <FACET 4.1> computational sequence ...


<Success>: <FACET 4.1> computational sequence data in correct format.
<Status>: Checking the integrity of the metadata in <FACET 4.1> computational sequence ...
<Success>: <FACET 4.1> computational sequence metadata in correct format
<Success>: Computational sequence read from file ./data/CMU_MOSI_COVAREP.csd ...
<Status>: Checking the integrity of the data in <COAVAREP> computational sequence ...
<Success>: <COAVAREP> computational sequence data in correct format.
<Status>: Checking the integrity of the metadata in <COAVAREP> computational sequence ...
<Success>: <COAVAREP> computational sequence metadata in correct format
<Success>: Dataset initialized successfully ... 


## A peek into the dataset

The multimodal dataset, after loaded, has the following hierarchy:


```
            computational_sequence_1 ---...
           /                                   ...
          /                                    /
         /                          first_video     features -- T X N array
        /                          /               /
dataset ---computational_sequence_2 -- second_video
        \                          \               \
         \                          third_video     intervals -- T X 2 array
          \                                    \...
           \
            computational_sequence_3 ---...
```

It looks like a nested dictionary and can be indexed as if it is a nested dictionary. A dataset contains multiple computational sequences whose key is the `text_field`, `visual_field`, `acoustic_field` as defined above. Each computational sequence, however, has multiple video IDs in it, and different computational sequences are supposed to have the same set of video IDs. Within each video, there are two arrays: `features` and `intervals`, denoting the feature values at each time step and the start and end timestamp for each step. We can take a look at its content.

In [5]:
print(list(dataset.keys()))
print("=" * 80)

print(list(dataset[visual_field].keys())[:10])
print("=" * 80)

some_id = list(dataset[visual_field].keys())[15]
print(list(dataset[visual_field][some_id].keys()))
print("=" * 80)

print(list(dataset[visual_field][some_id]['intervals'].shape))
print("=" * 80)

print(list(dataset[visual_field][some_id]['features'].shape))
print(list(dataset[text_field][some_id]['features'].shape))
print(list(dataset[acoustic_field][some_id]['features'].shape))
print("Different modalities have different number of time steps!")

['CMU_MOSI_ModifiedTimestampedWords', 'CMU_MOSI_VisualFacet_4.1', 'CMU_MOSI_COVAREP']
['03bSnISJMiM', '0h-zjBukYpk', '1DmNV9C1hbY', '1iG0909rllw', '2WGyTLYerpo', '2iD-tVS8NPw', '5W7Z1C_fDaE', '6Egk_28TtTM', '6_0THN4chvY', '73jzhE8R1TQ']
['features', 'intervals']
[5404, 2]
[5404, 47]
[658, 1]
[18009, 74]
Different modalities have different number of time steps!


## Append annotations to the dataset and get the data points

Now that we have a preprocessed dataset, all we need to do is to apply annotations to the data. Annotations are also computational sequences, since they are also just some values distributed on different time spans (e.g 1-3s is 'angry', 12-26s is 'neutral'). Hence, we just add the label computational sequence to the dataset and then align to the labels. Since we (may) want to preserve the whole sequences, this time we don't specify any collapse functions when aligning. 

Note that after alignment, the keys in the dataset changes from `video_id` to `video_id[segment_no]`, because alignment will segment each datapoint based on the segmentation of the pivot modality (in this case, it is segmented based on labels, which is what we need, and yes, one code block ago they are segmented to word level, which I didn't show you).

***Important: DO NOT add the labels together at the beginning, the labels will be segmented during the first alignment to words. This also holds for any situation where you want to do multiple levels of alignment.***

In [40]:
label_field = 'CMU_MOSI_Opinion_Labels'

# we add and align to lables to obtain labeled segments
# this time we don't apply collapse functions so that the temporal sequences are preserved
label_recipe = {label_field: os.path.join(DATA_PATH, label_field + '.csd')}
dataset.add_computational_sequences(label_recipe, destination=None)

<Success>: Computational sequence read from file ./data/CMU_MOSI_Opinion_Labels.csd ...
<Status>: Checking the integrity of the data in <Opinion Segment Labels> computational sequence ...
<Success>: <Opinion Segment Labels> computational sequence data in correct format.
<Status>: Checking the integrity of the metadata in <Opinion Segment Labels> computational sequence ...
<Success>: <Opinion Segment Labels> computational sequence metadata in correct format


In [7]:
aligned = dataset.align(label_field, replace=False)
# first we align to words with averaging, collapse_function receives a list of functions
aligned = aligned.align(text_field, replace=False)

No split [ version
<Status>: Unify was called ...
<Success>: Unify completed ...
<Status>: Alignment based on <CMU_MOSI_ModifiedTimestampedWords> computational sequence will start shortly ...


Aligning 03bSnISJMiM[0]:   0%|          | 0/8 [00:00<?, ? Segments/s]
                                                                     
Aligning 03bSnISJMiM[1]:   0%|          | 0/27 [00:00<?, ? Segments/s]
                                                                      
Aligning 03bSnISJMiM[2]:   0%|          | 0/10 [00:00<?, ? Segments/s]
                                                                      
Aligning 03bSnISJMiM[3]:   0%|          | 0/7 [00:00<?, ? Segments/s]
                                                                     
Aligning 03bSnISJMiM[4]:   0%|          | 0/5 [00:00<?, ? Segments/s]
                                                                     
Aligning 03bSnISJMiM[5]:   0%|          | 0/6 [00:00<?, ? Segments/s]
                                                                     
Aligning 03bSnISJMiM[7]:   0%|          | 0/20 [00:00<?, ? Segments/s]
                                                                      
Aligning 03bSn

Aligning 1DmNV9C1hbY[2]:   0%|          | 0/11 [00:00<?, ? Segments/s]
                                                                      
Aligning 1DmNV9C1hbY[3]:   0%|          | 0/21 [00:00<?, ? Segments/s]
                                                                      
Aligning 1DmNV9C1hbY[4]:   0%|          | 0/12 [00:00<?, ? Segments/s]
                                                                      
Aligning 1DmNV9C1hbY[5]:   0%|          | 0/18 [00:00<?, ? Segments/s]
                                                                      
Aligning 1DmNV9C1hbY[6]:   0%|          | 0/14 [00:00<?, ? Segments/s]
                                                                      
Aligning 1DmNV9C1hbY[8]:   0%|          | 0/10 [00:00<?, ? Segments/s]
                                                                      
Aligning 1DmNV9C1hbY[9]:   0%|          | 0/7 [00:00<?, ? Segments/s]
                                                                     
Aligning

Aligning 1iG0909rllw[29]:   0%|          | 0/7 [00:00<?, ? Segments/s]
                                                                      
Aligning 2WGyTLYerpo[1]:   0%|          | 0/6 [00:00<?, ? Segments/s]
                                                                     
Aligning 2WGyTLYerpo[2]:   0%|          | 0/22 [00:00<?, ? Segments/s]
                                                                      
Aligning 2WGyTLYerpo[3]:   0%|          | 0/15 [00:00<?, ? Segments/s]
                                                                      
Aligning 2WGyTLYerpo[4]:   0%|          | 0/16 [00:00<?, ? Segments/s]
                                                                      
Aligning 2WGyTLYerpo[5]:   0%|          | 0/10 [00:00<?, ? Segments/s]
                                                                      
Aligning 2WGyTLYerpo[6]:   0%|          | 0/6 [00:00<?, ? Segments/s]
                                                                     
Aligning 2

                                                                      
Aligning 2WGyTLYerpo[40]:   0%|          | 0/10 [00:00<?, ? Segments/s]
                                                                       
Aligning 2WGyTLYerpo[41]:   0%|          | 0/6 [00:00<?, ? Segments/s]
                                                                      
Aligning 2WGyTLYerpo[44]:   0%|          | 0/8 [00:00<?, ? Segments/s]
                                                                      
Aligning 2WGyTLYerpo[45]:   0%|          | 0/5 [00:00<?, ? Segments/s]
                                                                      
Aligning 2WGyTLYerpo[46]:   0%|          | 0/3 [00:00<?, ? Segments/s]
                                                                      
Aligning 2WGyTLYerpo[47]:   0%|          | 0/5 [00:00<?, ? Segments/s]
                                                                      
Aligning 2WGyTLYerpo[48]:   0%|          | 0/8 [00:00<?, ? Segments/s]
    

...

Aligning wMbj6ajWbic[23]:   0%|          | 0/8 [00:00<?, ? Segments/s]
                                                                      
Aligning wMbj6ajWbic[24]:   0%|          | 0/11 [00:00<?, ? Segments/s]
                                                                       
Aligning wMbj6ajWbic[25]:   0%|          | 0/10 [00:00<?, ? Segments/s]
                                                                       
Aligning wMbj6ajWbic[26]:   0%|          | 0/7 [00:00<?, ? Segments/s]
                                                                      
Aligning wMbj6ajWbic[28]:   0%|          | 0/17 [00:00<?, ? Segments/s]
                                                                       
Aligning wMbj6ajWbic[29]:   0%|          | 0/13 [00:00<?, ? Segments/s]
                                                                       
Aligning yDtzw_Y-7RU[0]:   0%|          | 0/17 [00:00<?, ? Segments/s]
                                                                     

Aligning yvsjCA6Y5Fc[8]:   0%|          | 0/18 [00:00<?, ? Segments/s]
                                                                      
Aligning yvsjCA6Y5Fc[9]:   0%|          | 0/23 [00:00<?, ? Segments/s]
                                                                      
Aligning yvsjCA6Y5Fc[10]:   0%|          | 0/13 [00:00<?, ? Segments/s]
                                                                       
Aligning yvsjCA6Y5Fc[11]:   0%|          | 0/40 [00:00<?, ? Segments/s]
                                                                       
Aligning yvsjCA6Y5Fc[12]:   0%|          | 0/11 [00:00<?, ? Segments/s]
                                                                       
Aligning yvsjCA6Y5Fc[14]:   0%|          | 0/14 [00:00<?, ? Segments/s]
                                                                       
Aligning yvsjCA6Y5Fc[15]:   0%|          | 0/51 [00:00<?, ? Segments/s]
                                                                    

                                                                       
Aligning zhpQhgha_KU[26]:   0%|          | 0/7 [00:00<?, ? Segments/s]
                                                                      
Aligning zhpQhgha_KU[27]:   0%|          | 0/13 [00:00<?, ? Segments/s]
                                                                       
Aligning zhpQhgha_KU[29]:   0%|          | 0/13 [00:00<?, ? Segments/s]
                                                                       
Aligning zhpQhgha_KU[30]:   0%|          | 0/26 [00:00<?, ? Segments/s]
                                                                       
Aligning zhpQhgha_KU[31]:   0%|          | 0/21 [00:00<?, ? Segments/s]
                                                                       
Aligning zhpQhgha_KU[32]:   0%|          | 0/12 [00:00<?, ? Segments/s]
                                                                       
Aligning zhpQhgha_KU[33]:   0%|          | 0/7 [00:00<?, ? Segment

<Success>: Alignment to <CMU_MOSI_ModifiedTimestampedWords> complete.
<Status>: Replacing dataset content with aligned computational sequences
<Success>: Initialized empty <CMU_MOSI_ModifiedTimestampedWords> computational sequence.
<Status>: Checking the integrity of the data in <CMU_MOSI_ModifiedTimestampedWords> computational sequence ...
<Success>: <CMU_MOSI_ModifiedTimestampedWords> computational sequence data in correct format.
<Status>: Checking the integrity of the metadata in <CMU_MOSI_ModifiedTimestampedWords> computational sequence ...
<Success>: <CMU_MOSI_ModifiedTimestampedWords> computational sequence metadata in correct format
<Success>: Initialized empty <CMU_MOSI_VisualFacet_4.1> computational sequence.
<Status>: Checking the integrity of the data in <CMU_MOSI_VisualFacet_4.1> computational sequence ...
<Success>: <CMU_MOSI_VisualFacet_4.1> computational sequence data in correct format.
<Status>: Checking the integrity of the metadata in <CMU_MOSI_VisualFacet_4.1> compu

<Success>: <CMU_MOSI_COVAREP> computational sequence data in correct format.
<Status>: Checking the integrity of the metadata in <CMU_MOSI_COVAREP> computational sequence ...
<Success>: <CMU_MOSI_COVAREP> computational sequence metadata in correct format
<Success>: Initialized empty <CMU_MOSI_Opinion_Labels> computational sequence.
<Status>: Checking the integrity of the data in <CMU_MOSI_Opinion_Labels> computational sequence ...
<Success>: <CMU_MOSI_Opinion_Labels> computational sequence data in correct format.
<Status>: Checking the integrity of the metadata in <CMU_MOSI_Opinion_Labels> computational sequence ...
<Success>: <CMU_MOSI_Opinion_Labels> computational sequence metadata in correct format
03bSnISJMiM[4][3]


In [15]:
label_4_keys = [k for k in list(dataset[text_field].keys()) if k.startswith('03bSnISJMiM[4]')]
print('\n'.join([str(dataset[label_field][k]) for k in label_4_keys]))
print()
print('\n'.join([str(dataset[text_field][k]) for k in label_4_keys]))
print()
print('\n'.join([str(dataset[visual_field][k]) for k in label_4_keys]))
print()

{'intervals': array([[71.07098 , 71.160774]], dtype=float32), 'features': array([[0.]], dtype=float32)}
{'intervals': array([[71.160774, 71.34036 ]], dtype=float32), 'features': array([[0.]], dtype=float32)}
{'intervals': array([[71.34036, 71.51995]], dtype=float32), 'features': array([[0.]], dtype=float32)}
{'intervals': array([[71.51995, 71.69955]], dtype=float32), 'features': array([[0.]], dtype=float32)}
{'intervals': array([[71.69955 , 71.829254]], dtype=float32), 'features': array([[0.]], dtype=float32)}

{'intervals': array([[71.07097626, 71.16077098]]), 'features': array([[b'and']], dtype='|S12')}
{'intervals': array([[71.16077098, 71.34036281]]), 'features': array([[b'they']], dtype='|S12')}
{'intervals': array([[71.34036281, 71.51995465]]), 'features': array([[b'shouldve']], dtype='|S12')}
{'intervals': array([[71.51995465, 71.69954649]]), 'features': array([[b'i']], dtype='|S12')}
{'intervals': array([[71.69954649, 71.8292517 ]]), 'features': array([[b'guess']], dtype='|S12'

In [49]:
def strictly_increasing(L):
    return all(x<y for x, y in zip(L, L[1:]))
for vid_id in dataset[text_field].keys():
    label_id = -1
    word_id = 0
    words = []
    word_features = []
    word_intervals = []
    original_vid = dataset[text_field][vid_id]
    feature_interval = np.concatenate([original_vid['features'], original_vid['intervals']], axis=1)
    if '%s[0][0]' % (vid_id) not in aligned[label_field].keys():
        print('=======' + vid_id)
    while True:
        label_id += 1
        if '%s[%d][%d]' % (vid_id, label_id, word_id) not in aligned[text_field].keys():
            break
        while True:
            aligned_vid_id = '%s[%d][%d]' % (vid_id, label_id, word_id)
            if aligned_vid_id in aligned[text_field].keys():
                assert aligned[text_field][aligned_vid_id]['features'].shape == (1, 1)
                word = aligned[text_field][aligned_vid_id]['features'][0][0].decode('utf-8')
                if word == 'sp':
                    word = ' '
                words.append(word)
                word_features.append(aligned[text_field][aligned_vid_id]['features'])
                word_intervals.append(aligned[text_field][aligned_vid_id]['intervals'])
                word_id += 1
            else:
                break
    print(' '.join(words))
    features_array = np.concatenate(word_features, axis=0)
    intervals_array = np.concatenate(word_intervals, axis=0)
    assert strictly_increasing(intervals_array[0,:])
    assert strictly_increasing(intervals_array[1,:])
    labeled_feature_interval = np.concatenate([features_array, intervals_array], axis=1)
    print()

  anyhow   it was   really good whole   bunch of   background info on   why   she has to   fight and be prepared
  now the title of the movie basically says it all movie frustrated me to such an extreme extent   that i was loudly exclaiming why   at the end of the film
but for this one   i just didnt feel like it the   chipmunks movie
and   i thought they were average not master pieces actually really rather enjoyable
say that i really enjoyed cowboys and aliens
  before i went in to see the green lantern ive been hearing all these like critics basically just trashing this movie   left and right
  probably one of the ballsiest probably   one of the ballsiest fims od sundance   2011 um
  i just got finished watching an excellent movie called mars needs moms be   milo   milos mother   and gribble
and so i was kind of curious to see   maybe if that was what disney was going for and it definately was what thay were going for able to really   tell what the difference is
  um   so   there ar

In [30]:
np.array([aligned[text_field]['03bSnISJMiM[4][0]']['features'][0][0].decode('utf-8'),
                aligned[text_field]['03bSnISJMiM[4][1]']['features'][0][0].decode('utf-8')])

array(['and', 'they'], dtype='<U4')

In [32]:
aligned[text_field]['03bSnISJMiM[4][0]']['features'].shape == (1, 1)

True

In [52]:
dataset[text_field]['6_0THN4chvY']['intervals']

dict_keys(['03bSnISJMiM', '0h-zjBukYpk', '1DmNV9C1hbY', '1iG0909rllw', '2WGyTLYerpo', '2iD-tVS8NPw', '5W7Z1C_fDaE', '6Egk_28TtTM', '6_0THN4chvY', '73jzhE8R1TQ', '7JsX8y1ysxY', '8OtFthrtaJM', '8d-gEyoeBzc', '8qrpnFRGt2A', '9J25DZhivz8', '9T9Hf74oK10', '9c67fiY0wGQ', '9qR7uwkblbs', 'Af8D0E4ZXaw', 'BI97DNYfe5I', 'BXuRRbG0Ugk', 'Bfr499ggo-0', 'BioHAh1qJAQ', 'BvYR0L6f2Ig', 'Ci-AH39fi3Y', 'Clx4VXItLTE', 'Dg_0XKD0Mf4', 'G-xst2euQUc', 'G6GlGvlkxAQ', 'GWuJjcEuzt8', 'HEsqda8_d0Q', 'I5y0__X72p0', 'Iu2PFX3z_1s', 'IumbAb8q2dM', 'Jkswaaud0hk', 'LSi-o-IrDMs', 'MLal-t_vJPM', 'Njd1F0vZSm4', 'Nzq88NnDkEk', 'OQvJTdtJ2H4', 'OtBXNcAL_lE', 'Oz06ZWiO20M', 'POKffnXeBds', 'PZ-lDQFboO8', 'QN9ZIUWUXsY', 'Qr1Ca94K55A', 'Sqr0AcuoNnk', 'TvyZBvOMOTc', 'VCslbP0mgZI', 'VbQk4H8hgr0', 'Vj1wYRQjB-o', 'W8NXH0Djyww', 'WKA5OygbEKI', 'X3j2zQgwYgE', 'ZAIRrfG22O0', 'ZUXBRvtny7o', '_dI--eQ6qVU', 'aiEXnCPZubE', 'atnd_PF-Lbs', 'bOL9jKpeJRs', 'bvLlb-M3UXU', 'c5xsKMxpXnc', 'c7UH_rxdZv4', 'cM3Yna7AavY', 'cW1FSBF59ik', 'cXypl4FnoZo',

In [48]:
'7JsX8y1ysxY[0][0]' in aligned[text_field].keys()

True

In [54]:
np.concatenate([dataset[text_field]['7JsX8y1ysxY']['features'], dataset[text_field]['7JsX8y1ysxY']['intervals']], axis=1)

array([[b'all', b'0.48140589569200004', b'0.5712018140590001'],
       [b'right', b'0.5712018140590001', b'0.601133786848'],
       [b'i', b'0.601133786848', b'0.7308390022679999'],
       ...,
       [b'to', b'178.64648526099998', b'179.424716553'],
       [b'sp', b'179.424716553', b'179.913605442'],
       [b'say', b'179.913605442', b'179.993424036']], dtype='|S32')

## Splitting the dataset

Now it comes to our final step: splitting the dataset into train/dev/test splits. This code block is a bit long in itself, so be patience and step through carefully with the explanatory comments.

The SDK provides the splits in terms of video IDs (which video belong to which split), however, after alignment our dataset keys already changed from `video_id` to `video_id[segment_no]`. Hence, we need to extract the video ID when looping through the data to determine which split each data point belongs to.

In the following data processing, I also include instance-wise Z-normalization (subtract by mean and divide by standard dev) and converted words to unique IDs.

This example is based on PyTorch so I am using PyTorch related utils, but the same procedure should be easy to adapt to other frameworks.

In [8]:
# obtain the train/dev/test splits - these splits are based on video IDs
train_split = DATASET.standard_folds.standard_train_fold
dev_split = DATASET.standard_folds.standard_valid_fold
test_split = DATASET.standard_folds.standard_test_fold

# inspect the splits: they only contain video IDs
print(test_split)

['tmZoasNr4rU', 'zhpQhgha_KU', 'lXPQBPVc5Cw', 'iiK8YX8oH1E', 'tStelxIAHjw', 'nzpVDcQ0ywM', 'etzxEpPuc6I', 'cW1FSBF59ik', 'd6hH302o4v8', 'k5Y_838nuGo', 'pLTX3ipuDJI', 'jUzDDGyPkXU', 'f_pcplsH_V0', 'yvsjCA6Y5Fc', 'nbWiPyCm4g0', 'rnaNMUZpvvg', 'wMbj6ajWbic', 'cM3Yna7AavY', 'yDtzw_Y-7RU', 'vyB00TXsimI', 'dq3Nf_lMPnE', 'phBUpBr1hSo', 'd3_k5Xpfmik', 'v0zCBqDeKcE', 'tIrG4oNLFzE', 'fvVhgmXxadc', 'ob23OKe5a9Q', 'cXypl4FnoZo', 'vvZ4IcEtiZc', 'f9O3YtZ2VfI', 'c7UH_rxdZv4']


In [21]:
# we can see they are in the format of 'video_id[segment_no]', but the splits was specified with video_id only
# we need to use regex or something to match the video IDs...
import torch
import torch.nn as nn

from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm_notebook
from collections import defaultdict

# a sentinel epsilon for safe division, without it we will replace illegal values with a constant
EPS = 0

# construct a word2id mapping that automatically takes increment when new words are encountered
word2id = defaultdict(lambda: len(word2id))
UNK = word2id['<unk>']
PAD = word2id['<pad>']

# place holders for the final train/dev/test dataset
train = []
dev = []
test = []

# define a regular expression to extract the video ID out of the keys
pattern = re.compile('(.*)\[.*\]')
num_drop = 0 # a counter to count how many data points went into some processing issues

for segment in dataset[label_field].keys():
    
    # get the video ID and the features out of the aligned dataset
    vid = re.search(pattern, segment).group(1)
    label = dataset[label_field][segment]['features']
    _words = dataset[text_field][segment]['features']
    _visual = dataset[visual_field][segment]['features']
    _acoustic = dataset[acoustic_field][segment]['features']

    # if the sequences are not same length after alignment, there must be some problem with some modalities
    # we should drop it or inspect the data again
    if not _words.shape[0] == _visual.shape[0] == _acoustic.shape[0]:
        print(f"Encountered datapoint {vid} with text shape {_words.shape}, visual shape {_visual.shape}, acoustic shape {_acoustic.shape}")
        num_drop += 1
        continue

    # remove nan values
    label = np.nan_to_num(label)
    _visual = np.nan_to_num(_visual)
    _acoustic = np.nan_to_num(_acoustic)

    # remove speech pause tokens - this is in general helpful
    # we should remove speech pauses and corresponding visual/acoustic features together
    # otherwise modalities would no longer be aligned
    words = []
    visual = []
    acoustic = []
    for i, word in enumerate(_words):
        if word[0] != b'sp':
            words.append(word2id[word[0].decode('utf-8')]) # SDK stores strings as bytes, decode into strings here
            visual.append(_visual[i, :])
            acoustic.append(_acoustic[i, :])

    words = np.asarray(words)
    visual = np.asarray(visual)
    acoustic = np.asarray(acoustic)

    # z-normalization per instance and remove nan/infs
    visual = np.nan_to_num((visual - visual.mean(0, keepdims=True)) / (EPS + np.std(visual, axis=0, keepdims=True)))
    acoustic = np.nan_to_num((acoustic - acoustic.mean(0, keepdims=True)) / (EPS + np.std(acoustic, axis=0, keepdims=True)))

    if vid in train_split:
        train.append(((words, visual, acoustic), label, segment))
    elif vid in dev_split:
        dev.append(((words, visual, acoustic), label, segment))
    elif vid in test_split:
        test.append(((words, visual, acoustic), label, segment))
    else:
        print(f"Found video that doesn't belong to any splits: {vid}")

print(f"Total number of {num_drop} datapoints have been dropped.")

# turn off the word2id - define a named function here to allow for pickling
def return_unk():
    return UNK
word2id.default_factory = return_unk

/media/bighdd7/zhun/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:67: RuntimeWarning: invalid value encountered in true_divide
/media/bighdd7/zhun/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:66: RuntimeWarning: invalid value encountered in true_divide


Encountered datapoint 5W7Z1C_fDaE with text shape (7, 1), visual shape (6, 47), acoustic shape (7, 74)
Encountered datapoint BioHAh1qJAQ with text shape (50, 1), visual shape (49, 47), acoustic shape (50, 74)
Encountered datapoint lXPQBPVc5Cw with text shape (27, 1), visual shape (26, 47), acoustic shape (27, 74)
Total number of 3 datapoints have been dropped.


/media/bighdd7/zhun/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:67: RuntimeWarning: divide by zero encountered in true_divide
/media/bighdd7/zhun/anaconda3/lib/python3.7/site-packages/numpy/core/_methods.py:75: RuntimeWarning: overflow encountered in reduce
  ret = umr_sum(arr, axis, dtype, out, keepdims)
/media/bighdd7/zhun/anaconda3/lib/python3.7/site-packages/numpy/core/_methods.py:107: RuntimeWarning: overflow encountered in reduce
  arrmean = umr_sum(arr, axis, dtype, keepdims=True)
/media/bighdd7/zhun/anaconda3/lib/python3.7/site-packages/numpy/core/_methods.py:121: RuntimeWarning: overflow encountered in multiply
  x = um.multiply(x, x, out=x)


## Inspect the dataset

Now that we have loaded the data, we can check the sizes of each split, data point shapes, vocabulary size, etc.

In [22]:
# let's see the size of each set and shape of data
print(len(train))
print(len(dev))
print(len(test))

print(train[0][0][1].shape)
print(train[0][1].shape)
print(train[0][1])

print(f"Total vocab size: {len(word2id)}")

1281
229
685
(5, 47)
(1, 1)
[[2.4]]
Total vocab size: 3117


## Collate function in PyTorch

Collate functions are functions used by PyTorch dataloader to gather batched data from dataset. It loads multiple data points from an iterable dataset object and put them in a certain format. Here we just use the lists we've constructed as the dataset and assume PyTorch dataloader will operate on that.

In [29]:
def multi_collate(batch):
    '''
    Collate functions assume batch = [Dataset[i] for i in index_set]
    '''
    # for later use we sort the batch in descending order of length
    batch = sorted(batch, key=lambda x: x[0][0].shape[0], reverse=True)
    
    # get the data out of the batch - use pad sequence util functions from PyTorch to pad things
    labels = torch.cat([torch.from_numpy(sample[1]) for sample in batch], dim=0)
    sentences = pad_sequence([torch.LongTensor(sample[0][0]) for sample in batch], padding_value=PAD)
    visual = pad_sequence([torch.FloatTensor(sample[0][1]) for sample in batch])
    acoustic = pad_sequence([torch.FloatTensor(sample[0][2]) for sample in batch])
    
    # lengths are useful later in using RNNs
    lengths = torch.LongTensor([sample[0][0].shape[0] for sample in batch])
    return sentences, visual, acoustic, labels, lengths

# construct dataloaders, dev and test could use around ~X3 times batch size since no_grad is used during eval
batch_sz = 56
train_loader = DataLoader(train, shuffle=True, batch_size=batch_sz, collate_fn=multi_collate)
dev_loader = DataLoader(dev, shuffle=False, batch_size=batch_sz*3, collate_fn=multi_collate)
test_loader = DataLoader(test, shuffle=False, batch_size=batch_sz*3, collate_fn=multi_collate)

# let's create a temporary dataloader just to see how the batch looks like
temp_loader = iter(DataLoader(test, shuffle=True, batch_size=8, collate_fn=multi_collate))
batch = next(temp_loader)

print(batch[0].shape) # word vectors, padded to maxlen
print(batch[1].shape) # visual features
print(batch[2].shape) # acoustic features
print(batch[3]) # labels
print(batch[4]) # lengths

torch.Size([28, 8])
torch.Size([28, 8, 47])
torch.Size([28, 8, 74])
tensor([[ 1.4000],
        [ 0.2000],
        [ 0.4000],
        [-0.8000],
        [-2.0000],
        [-2.0000],
        [-1.0000],
        [-0.6000]])
tensor([28, 19, 14, 13, 10,  6,  6,  6])


In [30]:
# Let's actually inspect the transcripts to ensure it's correct
id2word = {v:k for k, v in word2id.items()}
examine_target = train
idx = np.random.randint(0, len(examine_target))
print(' '.join(list(map(lambda x: id2word[x], examine_target[idx][0][0].tolist()))))
# print(' '.join(examine_target[idx][0]))
print(examine_target[idx][1])
print(examine_target[idx][2])

hes funny
[[2.]]
7JsX8y1ysxY[9]


## Define a multimodal model

Here we show a simple example of late-fusion LSTM. Late-fusion refers to combining the features from different modalities at the final prediction stage, without introducing any interactions between them before that.

In [31]:
# let's define a simple model that can deal with multimodal variable length sequence
class LFLSTM(nn.Module):
    def __init__(self, input_sizes, hidden_sizes, fc1_size, output_size, dropout_rate):
        super(LFLSTM, self).__init__()
        self.input_size = input_sizes
        self.hidden_size = hidden_sizes
        self.fc1_size = fc1_size
        self.output_size = output_size
        self.dropout_rate = dropout_rate
        
        # defining modules - two layer bidirectional LSTM with layer norm in between
        self.embed = nn.Embedding(len(word2id), input_sizes[0])
        self.trnn1 = nn.LSTM(input_sizes[0], hidden_sizes[0], bidirectional=True)
        self.trnn2 = nn.LSTM(2*hidden_sizes[0], hidden_sizes[0], bidirectional=True)
        
        self.vrnn1 = nn.LSTM(input_sizes[1], hidden_sizes[1], bidirectional=True)
        self.vrnn2 = nn.LSTM(2*hidden_sizes[1], hidden_sizes[1], bidirectional=True)
        
        self.arnn1 = nn.LSTM(input_sizes[2], hidden_sizes[2], bidirectional=True)
        self.arnn2 = nn.LSTM(2*hidden_sizes[2], hidden_sizes[2], bidirectional=True)

        self.fc1 = nn.Linear(sum(hidden_sizes)*4, fc1_size)
        self.fc2 = nn.Linear(fc1_size, output_size)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(dropout_rate)
        self.tlayer_norm = nn.LayerNorm((hidden_sizes[0]*2,))
        self.vlayer_norm = nn.LayerNorm((hidden_sizes[1]*2,))
        self.alayer_norm = nn.LayerNorm((hidden_sizes[2]*2,))
        self.bn = nn.BatchNorm1d(sum(hidden_sizes)*4)

        
    def extract_features(self, sequence, lengths, rnn1, rnn2, layer_norm):
        packed_sequence = pack_padded_sequence(sequence, lengths)
        packed_h1, (final_h1, _) = rnn1(packed_sequence)
        padded_h1, _ = pad_packed_sequence(packed_h1)
        normed_h1 = layer_norm(padded_h1)
        packed_normed_h1 = pack_padded_sequence(normed_h1, lengths)
        _, (final_h2, _) = rnn2(packed_normed_h1)
        return final_h1, final_h2

        
    def fusion(self, sentences, visual, acoustic, lengths):
        batch_size = lengths.size(0)
        sentences = self.embed(sentences)
        
        # extract features from text modality
        final_h1t, final_h2t = self.extract_features(sentences, lengths, self.trnn1, self.trnn2, self.tlayer_norm)
        
        # extract features from visual modality
        final_h1v, final_h2v = self.extract_features(visual, lengths, self.vrnn1, self.vrnn2, self.vlayer_norm)
        
        # extract features from acoustic modality
        final_h1a, final_h2a = self.extract_features(acoustic, lengths, self.arnn1, self.arnn2, self.alayer_norm)

        
        # simple late fusion -- concatenation + normalization
        h = torch.cat((final_h1t, final_h2t, final_h1v, final_h2v, final_h1a, final_h2a),
                       dim=2).permute(1, 0, 2).contiguous().view(batch_size, -1)
        return self.bn(h)

    def forward(self, sentences, visual, acoustic, lengths):
        batch_size = lengths.size(0)
        h = self.fusion(sentences, visual, acoustic, lengths)
        h = self.fc1(h)
        h = self.dropout(h)
        h = self.relu(h)
        o = self.fc2(h)
        return o

## Load pretrained embeddings

We define a function for loading pretrained word embeddings stored in GloVe-style file. Contextualized embeddings obviously cannot be stored and loaded this way, though.

In [32]:
# define a function that loads data from GloVe-like embedding files
# we will add tutorials for loading contextualized embeddings later
# 2196017 is the vocab size of GloVe here.

def load_emb(w2i, path_to_embedding, embedding_size=300, embedding_vocab=2196017, init_emb=None):
    if init_emb is None:
        emb_mat = np.random.randn(len(w2i), embedding_size)
    else:
        emb_mat = init_emb
    f = open(path_to_embedding, 'r')
    found = 0
    for line in tqdm_notebook(f, total=embedding_vocab):
        content = line.strip().split()
        vector = np.asarray(list(map(lambda x: float(x), content[-300:])))
        word = ' '.join(content[:-300])
        if word in w2i:
            idx = w2i[word]
            emb_mat[idx, :] = vector
            found += 1
    print(f"Found {found} words in the embedding file.")
    return torch.tensor(emb_mat).float()

## Training a model

Next we train a model. We use Adam with gradient clipping and weight decay for training, and our loss here is Mean Absolute Error (MOSI is a regression dataset). We exclude the embeddings from trainable computation graph to prevent overfitting. We also apply a early-stopping scheme with learning rate annealing based on validation loss.

In [34]:
from tqdm import tqdm_notebook
from torch.optim import Adam, SGD
from sklearn.metrics import accuracy_score

torch.manual_seed(123)
torch.cuda.manual_seed_all(123)

CUDA = torch.cuda.is_available()
MAX_EPOCH = 1000

text_size = 300
visual_size = 47
acoustic_size = 74

# define some model settings and hyper-parameters
input_sizes = [text_size, visual_size, acoustic_size]
hidden_sizes = [int(text_size * 1.5), int(visual_size * 1.5), int(acoustic_size * 1.5)]
fc1_size = sum(hidden_sizes) // 2
dropout = 0.25
output_size = 1
curr_patience = patience = 8
num_trials = 3
grad_clip_value = 1.0
weight_decay = 0.1

if os.path.exists(CACHE_PATH):
    pretrained_emb, word2id = torch.load(CACHE_PATH)
elif WORD_EMB_PATH is not None:
    pretrained_emb = load_emb(word2id, WORD_EMB_PATH)
    torch.save((pretrained_emb, word2id), CACHE_PATH)
else:
    pretrained_emb = None

model = LFLSTM(input_sizes, hidden_sizes, fc1_size, output_size, dropout)
if pretrained_emb is not None:
    model.embed.weight.data = pretrained_emb
model.embed.requires_grad = False
optimizer = Adam([param for param in model.parameters() if param.requires_grad], weight_decay=weight_decay)

if CUDA:
    model.cuda()
criterion = nn.L1Loss(reduction='sum')
criterion_test = nn.L1Loss(reduction='sum')
best_valid_loss = float('inf')
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.1)
lr_scheduler.step() # for some reason it seems the StepLR needs to be stepped once first
train_losses = []
valid_losses = []
for e in range(MAX_EPOCH):
    model.train()
    train_iter = tqdm_notebook(train_loader)
    train_loss = 0.0
    for batch in train_iter:
        model.zero_grad()
        t, v, a, y, l = batch
        batch_size = t.size(0)
        if CUDA:
            t = t.cuda()
            v = v.cuda()
            a = a.cuda()
            y = y.cuda()
            l = l.cuda()
        y_tilde = model(t, v, a, l)
        loss = criterion(y_tilde, y)
        loss.backward()
        torch.nn.utils.clip_grad_value_([param for param in model.parameters() if param.requires_grad], grad_clip_value)
        optimizer.step()
        train_iter.set_description(f"Epoch {e}/{MAX_EPOCH}, current batch loss: {round(loss.item()/batch_size, 4)}")
        train_loss += loss.item()
    train_loss = train_loss / len(train)
    train_losses.append(train_loss)
    print(f"Training loss: {round(train_loss, 4)}")

    model.eval()
    with torch.no_grad():
        valid_loss = 0.0
        for batch in dev_loader:
            model.zero_grad()
            t, v, a, y, l = batch
            if CUDA:
                t = t.cuda()
                v = v.cuda()
                a = a.cuda()
                y = y.cuda()
                l = l.cuda()
            y_tilde = model(t, v, a, l)
            loss = criterion(y_tilde, y)
            valid_loss += loss.item()
    
    valid_loss = valid_loss/len(dev)
    valid_losses.append(valid_loss)
    print(f"Validation loss: {round(valid_loss, 4)}")
    print(f"Current patience: {curr_patience}, current trial: {num_trials}.")
    if valid_loss <= best_valid_loss:
        best_valid_loss = valid_loss
        print("Found new best model on dev set!")
        torch.save(model.state_dict(), 'model.std')
        torch.save(optimizer.state_dict(), 'optim.std')
        curr_patience = patience
    else:
        curr_patience -= 1
        if curr_patience <= -1:
            print("Running out of patience, loading previous best model.")
            num_trials -= 1
            curr_patience = patience
            model.load_state_dict(torch.load('model.std'))
            optimizer.load_state_dict(torch.load('optim.std'))
            lr_scheduler.step()
            print(f"Current learning rate: {optimizer.state_dict()['param_groups'][0]['lr']}")
    
    if num_trials <= 0:
        print("Running out of patience, early stopping.")
        break

model.load_state_dict(torch.load('model.std'))
y_true = []
y_pred = []
model.eval()
with torch.no_grad():
    test_loss = 0.0
    for batch in test_loader:
        model.zero_grad()
        t, v, a, y, l = batch
        if CUDA:
            t = t.cuda()
            v = v.cuda()
            a = a.cuda()
            y = y.cuda()
            l = l.cuda()
        y_tilde = model(t, v, a, l)
        loss = criterion_test(y_tilde, y)
        y_true.append(y_tilde.detach().cpu().numpy())
        y_pred.append(y.detach().cpu().numpy())
        test_loss += loss.item()
print(f"Test set performance: {test_loss/len(test)}")
y_true = np.concatenate(y_true, axis=0)
y_pred = np.concatenate(y_pred, axis=0)
                  
y_true_bin = y_true >= 0
y_pred_bin = y_pred >= 0
bin_acc = accuracy_score(y_true_bin, y_pred_bin)
print(f"Test set accuracy is {bin_acc}")

Training loss: 1.1386
Validation loss: 1.371
Current patience: 8, current trial: 3.
Found new best model on dev set!


Training loss: 0.8447
Validation loss: 1.2764
Current patience: 8, current trial: 3.
Found new best model on dev set!


Training loss: 0.7315
Validation loss: 1.1768
Current patience: 8, current trial: 3.
Found new best model on dev set!


Training loss: 0.6246
Validation loss: 1.2723
Current patience: 8, current trial: 3.


Training loss: 0.5839
Validation loss: 1.1591
Current patience: 7, current trial: 3.
Found new best model on dev set!


...

Training loss: 0.2406
Validation loss: 1.0407
Current patience: 4, current trial: 1.


Training loss: 0.2243
Validation loss: 1.0435
Current patience: 3, current trial: 1.


Training loss: 0.2372
Validation loss: 1.0399
Current patience: 2, current trial: 1.


Training loss: 0.2314
Validation loss: 1.0417
Current patience: 1, current trial: 1.


Training loss: 0.2231
Validation loss: 1.0375
Current patience: 0, current trial: 1.
Running out of patience, loading previous best model.
Current learning rate: 1.0000000000000002e-06
Running out of patience, early stopping.
Test set performance: 1.0627462150406664
Test set accuracy is 0.7167883211678832
